In [1]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
     

import torchvision
import torchvision.transforms as transforms

#from models import densenet as dn
from models import *

#import densenet as dn


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 1
model_name = "densenet_proj"
model = densenet()

print(model)


normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


=> Building model...
DenseNet3(
  (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (block1): DenseBlock(
    (layer): Sequential(
      (0): BottleneckBlock(
        (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv1): QuantConv2d(
          24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False
          (weight_quant): weight_quantize_fn()
        )
        (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): QuantConv2d(
          48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (weight_quant): weight_quantize_fn()
        )
      )
      (1): BottleneckBlock(
        (bn1): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv1): QuantConv2d(
          36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# This cell is from the website

lr = 6e-2
weight_decay = 1e-5
epochs = 400
best_prec = 92

model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.95, weight_decay=weight_decay)
# weight decay: for regularization to prevent overfitting


if not os.path.exists('result'):
    os.makedirs('result')
    
fdir = 'result/'+str(model_name)

if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)
    

In [2]:
fdir = 'result/'+str(model_name)
PATH = fdir+"/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9222/10000 (92%)



In [3]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, QuantConv2d):
        #print(layer)
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################
device = torch.device("cuda" if use_gpu else "cpu")
dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)
print("3rd convolution's input size:", save_output.outputs[1][0].size())
#print("2nd convolution's input size:", save_output.outputs[1][0].size())

3rd convolution's input size: torch.Size([1, 48, 32, 32])


In [ ]:
list(model.modules())[2].layer[0].conv2

In [ ]:
save_output.outputs[1][0].size()

In [4]:
w_bit = 4
weight_q = list(model.modules())[2].layer[0].conv2.weight_q # quantized value is stored during the training
w_alpha = list(model.modules())[2].layer[0].conv2.weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
#print(weight_int) # you should see clean integer numbers

In [5]:
x_bit = 4
x = save_output.outputs[1][0]  # input of the 2nd conv layer
x_alpha  = list(model.modules())[2].layer[0].conv2.act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
#print(x_int) # you should see clean integer numbers 

In [6]:
conv_int = torch.nn.Conv2d(in_channels = 48, out_channels=12, kernel_size = 3, padding = 1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)
output_recovered = output_int*w_delta*x_delta
#print(output_recovered)

In [ ]:
output_int.size()

In [ ]:
i = 0
for layer in model.modules():
    if(i==1 or i==2): print(layer.forward)
    i+=1

In [ ]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output_bn = SaveOutput()

for layer in model.modules():
    if isinstance(layer, nn.BatchNorm2d):
        print(layer)
        layer.register_forward_pre_hook(save_output_bn)       ## Input for the module will be grapped       
####################################################
device = torch.device("cuda" if use_gpu else "cpu")
dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)
print("2 bn's input size:", save_output_bn.outputs[1][0].size())

In [ ]:
#input to first bn
bn_1 = save_output_bn.outputs[0][0]

#input to bn after conv2
bn_3 = save_output_bn.outputs[2][0]

#output_final = output_recovered + bn_1
print(bn_3.size())
#only first 12 channels
bn_12 = bn_3[:,0:12,:,:]
abs(bn_12-output_recovered).mean()

In [ ]:
output_int.size()
weight_int.size()

In [7]:
a_int = x_int[0,:,:,:]
w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
                     
padding = 1
stride = 1
array_size_r = 48
array_size_c = 12
# row and column number

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

ic_tileg = range(1)
oc_tileg = range(1)

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(nig)+padding*2).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
# a_pad.size() = [64, (32+2pad)*(32+2pad)]


a_tile = torch.zeros(len(ic_tileg), array_size_r,    a_pad.size(1)).cuda() 
w_tile = torch.zeros(len(oc_tileg)*len(ic_tileg), array_size_c, array_size_r, len(kijg)).cuda() 

for ic_tile in ic_tileg:
    a_tile[ic_tile,:,:] = a_pad[ic_tile*array_size_r:(ic_tile+1)*array_size_r,:]

for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        w_tile[oc_tile*len(oc_tileg) + ic_tile,:,:,:] = w_int[oc_tile*array_size_c:(oc_tile+1)*array_size_c, ic_tile*array_size_r:(ic_tile+1)*array_size_r, :]


p_nijg = range(a_pad.size(1)) ## psum nij group

psum = torch.zeros(len(ic_tileg), len(oc_tileg), array_size_c, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:
    for ic_tile in ic_tileg:       # Tiling into array_sizeXarray_size array
        for oc_tile in oc_tileg:   # Tiling into array_sizeXarray_size array        
            for nij in p_nijg:       # time domain, sequentially given input
                    m = nn.Linear(array_size_c, array_size_r, bias=False)
                    #m.weight = torch.nn.Parameter(w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, kij])
                    m.weight = torch.nn.Parameter(w_tile[len(oc_tileg)*oc_tile+ic_tile,:,:,kij])
                    psum[ic_tile, oc_tile, :, nij, kij] = m(a_tile[ic_tile,:,nij]).cuda()

In [8]:
a_tile.size()
mod_a = a_tile.view(48,34,34)
mod_a = mod_a[:,0:3,0:18]
mod_a = torch.reshape(mod_a, (mod_a.size(0), -1))
mod_a.size()

torch.Size([48, 54])

In [9]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1)
o_nijg = range(o_ni_dim**2)    
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:
        for ic_tile in ic_tileg:    
            for oc_tile in oc_tileg:   
                out[oc_tile*array_size_c:(oc_tile+1)*array_size_c, o_nij] = out[oc_tile*array_size_c:(oc_tile+1)*array_size_c, o_nij] + \
                psum[ic_tile, oc_tile, :, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 4th index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

In [10]:
out_2D = torch.reshape(out, (out.size(0), o_ni_dim, -1))
difference = (out_2D - output_int[0,:,:,:])
print(difference.sum())

tensor(0.0131, device='cuda:0', grad_fn=<SumBackward0>)


In [14]:
X = mod_a[:,:]
bit_precision = 4
file = open('activation_d.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(int(X[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    


In [ ]:
### Complete this cell ###
tile_id = 0 
kij = 0
W = w_tile[tile_id,:,:,kij]  # w_tile[tile_num, array col num, array row num, kij]


bit_precision = 4
for p in range(9):
    W = w_tile[tile_id,:,:,p]
    file = open('weight_d_'+str(p)+'.txt', 'w') #write to file
    file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
    file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
    file.write('#................#\n')

    for i in range(W.size(0)):  # col time
        for j in range(W.size(1)): # row #
            if(W[i,7-j].item()<0):
                W_bin = '{0:04b}'.format(int(W[i,7-j].item()-0.001)+2**4)
                #print(W_bin)
            else:
                W_bin = '{0:04b}'.format(int(W[i,7-j].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file   

In [ ]:
out.size()

In [15]:
out_tile = out #out_R[:,:]  
#out_tile = REL(out_tile)
# psum[len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)]


bit_precision = 16
file = open('out_D.txt', 'w') #write to file
file.write('#nij0col7[msb-lsb],nij0col6[msb-lst],....,nij0col0[msb-lst]#\n')
file.write('#nij1col7[msb-lsb],nij1col6[msb-lst],....,nij1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(out_tile.size(1)):  # time step
    for j in range(out_tile.size(0)): # col #
        if(out_tile[7-j,i]<0):
            out_tile[7-j,i] = out_tile[7-j,i] + 65536
            out_bin = '{0:016b}'.format(int(out_tile[7-j,i].item()+0.001))
        else:
            out_bin = '{0:016b}'.format(int(out_tile[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(out_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file

In [17]:
### Complete this cell ###
ic_tile_id = 0 
oc_tile_id = 0 


kij = 0
nij = 0
P = psum[ic_tile_id,oc_tile_id,:,nij:nij+64,kij]  
# psum[len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)]
P.size(0)

bit_precision = 4
file = open('psum_d_0.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(P.size(1)):  # nij = time
    for j in range(P.size(0)): # col #
        if(P[7-j,i].item()<0):
            P_bin = '{0:04x}'.format(int(P[7-j,i].item()-0.001)+2**16)
            print(P_bin)
        else:
            P_bin = '{0:04x}'.format(int(P[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(P_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

fff4
fe36
ffaa
ff23
ffe5
ff5c
fff3
ff32
ffe1
ffa0
fff6
ffde
ffef
ff6f
ffbe
fff3
ff32
ffe1
ffa0
fff6
ffde
ffef
ff6f
ffbe
fff3
ff32
ffe1
ffa0
fff6
ffde
ffef
ff6f
ffbe
ffed
ff3b
ffdb
ff97
fff0
ffe4
ffe0
ff6e
ffbd
ffed
ff3b
ffdb
ff97
fff0
ffe4
ffe0
ff6e
ffbd
ffed
ff3b
ffdb
ff97
fff0
ffe4
ffe0
ff6e
ffbd
ffed
ff3b
ffdb
ff97
fff0
ffe4
ffe0
ff6e
ffbd
ffed
ff34
ffdb
ff8d
fff8
ffe6
ffe3
ff6a
ffc7
ffed
ff34
ffdb
ff8d
fff8
ffe6
ffe3
ff6a
ffc7
fff3
ff32
ffe1
ffa0
fff6
ffde
ffef
ff6f
ffbe
fff3
ff32
ffe1
ffa0
fff6
ffde
ffef
ff6f
ffbe
fff0
ff35
ffde
ff91
ffe8
ffec
fff7
ff76
ffd8
fff7
ff15
ffe6
ffcb
ffe2
ffcc
fffe
ff61
ff8a
ff34
ff5d
ffaa
ff60
ffa8
fffa
ff3f
fee7
ff8e
ffde
ffaa
ff73
ffad
ffcd
ffa3
ff8d
ffed
fee2
ffc3
ff5b
ff90
fff3
ff75
ffc9
ff6e
fffd
10000
ffb4
ff83
fff1
ff5d
ffd6
ff71
fffd
ffcc
ff95
ff4b
ffe1
ff8d
fffa
fff4
ffce
ff86
ffd9
fffa
ff53
ffd8
ff7c
fffb
ffc9
ff7d
ffe4
ffc4
ff12
ffe3
ffcc
ffbc
ff4a
ff3e
ff57
ffcf
ffa3
ff4e
ffbe
ffca
ff7c
ff54
ffd2
fff2
fff9
ff39
ffa3
ff01
ffed
ffec
ff97
fecb